In [2]:
import subprocess  # cli 실행
from pydub import AudioSegment
import math


def extract_audio_from_video(video_path, audio_path):
    command = ["ffmpeg", "-i", video_path, "-vn", audio_path]
    subprocess.run(command)


def split_audio(audio_path, chunks_folder, chunk_size=10):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = chunk_size * 60 * 1000
    chunks = math.ceil(len(track) / chunk_len)

    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i + 1) * chunk_len
        chunk = track[start_time:end_time]
        chunk.export(f"{chunks_folder}/chunk_{str(i).zfill(2)}.mp3", format="mp3")

In [4]:
split_audio(
    "files/audios/steve_jobs_keynotes_audio.mp3",
    "files/audios/chunks",
)

In [15]:
import openai
from glob import glob


def transcribe_chunks(chunk_folder, destination):
    files = sorted(glob(f"{chunk_folder}/*.mp3"))

    for file in files:
        with open(file, "rb") as audio_file, open(destination, "a") as text_file:
            transcript = openai.Audio.transcribe("whisper-1", audio_file)
            text_file.write(transcript["text"])


transcribe_chunks("./files/audios/chunks", "./files/audios/transcript.txt")